### Deploy a pre-trained Keras or TensorFlow model trained

Using Amazon SageMaker deploy and host pre-trained models from model h5 and json format


### Step 1. Set up

In the AWS Management Console, go to the Amazon SageMaker console. Choose Notebook Instances, and create a new notebook instance. Upload the current notebook and set the kernel to conda_tensorflow_p36.

The get_execution_role function retrieves the AWS Identity and Access Management (IAM) role you created at the time of creating your notebook instance.

In [1]:
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

### Step 2. Load the Keras model using the json and weights file
If you saved your model in the TensorFlow ProtoBuf format, skip to "Step 4. Convert the TensorFlow model to an Amazon SageMaker-readable format.

In [2]:
import keras
from keras.models import model_from_json

Using TensorFlow backend.


Create a directory called keras_model, navigate to keras_model from the Jupyter notebook home, and upload the model.json and model-weights.h5 files (using the "Upload" menu on the Jupyter notebook home).

In [3]:
#!mkdir keras_model

In [4]:
!ls keras_model

model.json  model-weights.h5


In [5]:
json_file = open('/home/ec2-user/SageMaker/sagemaker-keras/keras_model/'+'model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
loaded_model.load_weights('/home/ec2-user/SageMaker/sagemaker-keras/keras_model/model-weights.h5')
print("Loaded model from disk")


Loaded model from disk


### Step 3. Export the Keras model to the TensorFlow ProtoBuf format

In [7]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

In [8]:
# Note: This directory structure will need to be followed - see notes for the next section
model_version = '1'
export_dir = 'export/Servo/' + model_version

In [9]:
# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)

In [10]:
# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


In [11]:
from keras import backend as K

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


### Step 4. Convert TensorFlow model to a SageMaker readable format
Move the TensorFlow exported model into a directory export\Servo. SageMaker will recognize this as a loadable TensorFlow model. Your directory and file structure should look like:

In [12]:
!ls export

Servo


In [13]:
!ls export/Servo

1


In [14]:
!ls export/Servo/1

saved_model.pb	variables


In [15]:
!ls export/Servo/1/variables

variables.data-00000-of-00001  variables.index


In [16]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [17]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

### Step 5. Deploy the trained model

The entry_point file "train.py" needs to manually create before here. Otherwise gives error. this file can be an empty file to avoid error in below.

In [18]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '1.12',
                                  entry_point = 'train.py')

The Python 2 tensorflow images will be soon deprecated and may not be supported for newer upcoming versions of the tensorflow images.
Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.


In [20]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

Using already existing model: sagemaker-tensorflow-2019-12-03-19-49-13-841


ClientError: An error occurred (ValidationException) when calling the CreateEndpoint operation: Cannot create already existing endpoint "arn:aws:sagemaker:us-east-1:264117850417:endpoint/sagemaker-tensorflow-2019-12-03-19-49-13-841".

In [21]:
endpoint_name = 'sagemaker-tensorflow-2019-12-03-19-49-13-841'
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(endpoint_name, sagemaker_session)

### Step 6. Invoke the endpoint

In [22]:
import numpy as np

# The sample model expects an input of shape [1,50]
data = np.random.randn(1, 50)
predictor.predict(data)

{'outputs': {'score': {'dtype': 1,
   'tensor_shape': {'dim': [{'size': 1}, {'size': 1}]},
   'float_val': [0.5102317929267883]}},
 'model_spec': {'name': 'generic_model',
  'version': {'value': 1},
  'signature_name': 'serving_default'}}

Invoke the SageMaker endpoint using a boto3 client

In [25]:
import json
import boto3
import numpy as np
import io
 
client = boto3.client('runtime.sagemaker')
# The sample model expects an input of shape [1,50]
data = np.random.randn(1, 50).tolist()
response = client.invoke_endpoint(EndpointName=endpoint_name, Body=json.dumps(data))
print(data)
print("====req end ===")
response_body = response['Body']
print(response_body.read())

[[-0.633634592597317, 0.03817627703654733, -0.08634979463444686, 1.8877692693390504, -0.20845016913089398, 1.1958885810820756, -1.3904333478870916, -1.1622028071125237, 1.5852374363248427, -0.4944723149057209, 1.1948617962350085, -0.5104378049748406, -0.6651968545147159, -0.6397121775436853, -0.46724380108596, -0.19693134697746173, -0.7933309512260561, -0.4394124165808521, 2.814931088806231, 1.4820908897273357, 0.6343062371949156, 1.5263496792189206, 0.24243203324201576, 0.7383841134041175, -0.04157514991083119, -0.9488385393991836, 1.9576820515722528, 0.21978494940686033, -0.9462114443065387, -0.7776173019336995, 0.645442510535416, 0.015624843230571384, -1.6925402220165429, 1.1484579418337755, 1.54786242027252, 0.7813588531488627, -0.217348450895906, 1.2818648995423294, -1.0719787888954353, 0.013274864894420547, -1.6508206043876341, -0.03460607352014941, 1.0907161588530019, -2.405185611701825, -0.3674109532299552, 0.28104966540216186, -1.8247498761945533, -0.05785263833135548, 0.98007